In [ ]:
"""
This is an investigation code. It will be executed as a py file in CleanDatasetPackage folder.
"""

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# df1 is dataset for first month
df1 = pd.read_parquet('drive/MyDrive/data1.parquet')
df1.rename(columns={'airport_fee': 'Airport_fee'}, inplace=True)

In [30]:
start_timestamp = datetime(2023, 1, 1, 0, 0, 0)
end_timestamp = datetime(2023, 1, 31, 23, 59, 59)

# Create the timestamp column
df1['timestamp'] = df1['tpep_pickup_datetime'].apply(lambda x: start_timestamp + timedelta(minutes=((x - start_timestamp).total_seconds() // 600) * 10))

# Filter out timestamps outside of the desired range
df1 = df1[(df1['timestamp'] >= start_timestamp) & (df1['timestamp'] <= end_timestamp)]

### Deleting rows before 2022
because we want to predict data based on 2023 dataset

In [32]:
df1 = df1.query("tpep_pickup_datetime.dt.year == 2023 & tpep_dropoff_datetime.dt.year == 2023")

In [33]:
df1.set_index("tpep_pickup_datetime", inplace=True)

### getting columns with null values

In [48]:
df1.isnull().sum()

VendorID                     0
tpep_dropoff_datetime        0
passenger_count          71743
trip_distance                0
RatecodeID               71743
store_and_fwd_flag       71743
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71743
Airport_fee              71743
timestamp                    0
dtype: int64

### imputing passenger_count with median of passenger_count in each pickup zone and each back and next timestamp

these columns are empty together

In [49]:
df1.query("passenger_count.isna() & RatecodeID.isna() & store_and_fwd_flag.isnull() & payment_type == 0 & Airport_fee.isna()")

,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,timestamp
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,
2023-01-01 00:02:40,1,2023-01-01 00:30:36,NaN,0.00,NaN,None,142,79,0,20.13,0.0,0.5,0.00,0.0,1.0,24.13,NaN,NaN,2023-01-01 00:00:00
2023-01-01 00:57:48,1,2023-01-01 01:12:34,NaN,0.00,NaN,None,237,263,0,12.00,1.0,0.5,2.55,0.0,1.0,21.55,NaN,NaN,2023-01-01 00:50:00
2023-01-01 00:55:37,2,2023-01-01 01:00:41,NaN,0.86,NaN,None,24,41,0,12.80,0.0,0.5,2.14,0.0,1.0,16.44,NaN,NaN,2023-01-01 00:50:00
2023-01-01 00:43:47,2,2023-01-01 00:56:12,NaN,5.36,NaN,None,88,229,0,24.51,0.0,0.5,4.28,0.0,1.0,32.79,NaN,NaN,2023-01-01 00:40:00
2023-01-01 00:34:00,2,2023-01-01 00:51:00,NaN,6.03,NaN,None,232,263,0,32.30,0.0,0.5,7.26,0.0,1.0,43.56,NaN,NaN,2023-01-01 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 23:58:34,2,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.0,0.5,3.96,0.0,1.0,23.76,NaN,NaN,2023-01-31 23:50:00
2023-01-31 23:31:09,2,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.0,0.5,2.64,0.0,1.0,29.07,NaN,NaN,2023-01-31 23:30:00
2023-01-31 23:01:05,2,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.0,0.5,5.32,0.0,1.0,26.93,NaN,NaN,2023-01-31 23:00:00


trying to use correltaion to findout if passenger_count is realted to any other variable or not. Beacuse correlations are too low, we can see that passenger_count is missing completly at random(MCAR)

In [50]:
target_col = 'passenger_count'

corr_matrix = df1.corr()[target_col]

print(corr_matrix.sort_values(ascending=False))

<ipython-input-50-07b979b6bbfa>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df1.corr()[target_col]


passenger_count          1.000000
VendorID                 0.101141
fare_amount              0.032408
total_amount             0.030691
tolls_amount             0.024628
Airport_fee              0.012267
tip_amount               0.012015
congestion_surcharge     0.010283
payment_type             0.009472
improvement_surcharge    0.003894
trip_distance            0.002788
mta_tax                 -0.004493
DOLocationID            -0.005878
PULocationID            -0.011434
RatecodeID              -0.025028
extra                   -0.059212
Name: passenger_count, dtype: float64


In [51]:
def impute_passenger_count(x):
  missed_passengers = x[x.passenger_count.isna()]
  if len(missed_passengers) != 0:
    median_passengers = x.groupby('PULocationID')['passenger_count'].agg(np.median).reset_index()
    joined_x = pd.merge(missed_passengers, median_passengers, on='PULocationID', how='inner')
    x['passenger_count'].loc[x.passenger_count.isna()] = joined_x['passenger_count_y']
    x['passenger_count'].fillna(0, inplace=True)
  return x

In [52]:
df_imputed = df1.groupby(pd.Grouper(freq='10T')).apply(impute_passenger_count)

<ipython-input-52-ee011da2277c>:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_imputed = df1.groupby(pd.Grouper(freq='10T')).apply(impute_passenger_count)


**Compare summary statistics:** Calculate summary statistics such as mean, median, standard deviation, and variance for the original data and the imputed data, and compare them to ensure that they are similar. Large differences in these statistics could indicate bias in the imputed data.

In [53]:
df1.passenger_count.describe()

count    2.994975e+06
mean     1.362534e+00
std      8.961228e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.000000e+00
Name: passenger_count, dtype: float64

In [54]:
df_imputed.passenger_count.describe()

count    3.066718e+06
mean     1.330659e+00
std      9.092111e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.000000e+00
Name: passenger_count, dtype: float64

as we can see, std and mean of passenger_count before and after imputing missing values are quite similar

In [55]:
df1.passenger_count.value_counts()

1.0    2261360
2.0     451530
3.0     106352
4.0      53745
0.0      51164
5.0      42680
6.0      28124
8.0         13
7.0          6
9.0          1
Name: passenger_count, dtype: int64

In [56]:
df_imputed.passenger_count.value_counts()

1.0    2261360
2.0     451530
0.0     122907
3.0     106352
4.0      53745
5.0      42680
6.0      28124
8.0         13
7.0          6
9.0          1
Name: passenger_count, dtype: int64

### Find out columns which has 50% or more null values and drop them

In [57]:
null_percent = (df_imputed.isnull()).sum() / len(df_imputed) * 100

# Drop columns with 50% or more null values
drop_cols = null_percent[null_percent >= 50].index
df_imputed = df_imputed.drop(drop_cols, axis=1)

print(f"Dropped columns: {drop_cols}")

Dropped columns: Index([], dtype='object')


### imputing less important variables by its frequency or mode.
store_and_fwd_flag, retecodeid, congestion_surchage and airport_fee

In [58]:
mode_val = df_imputed["store_and_fwd_flag"].mode()[0]  # Calculate the mode of the column
df_imputed["store_and_fwd_flag"].fillna(mode_val, inplace=True)

mode_val = df_imputed["RatecodeID"].mode()[0]
df_imputed["RatecodeID"].fillna(mode_val, inplace=True)

In [59]:
df_imputed.congestion_surcharge.describe()

count    2.994975e+06
mean     2.274238e+00
std      7.718378e-01
min     -2.500000e+00
25%      2.500000e+00
50%      2.500000e+00
75%      2.500000e+00
max      2.500000e+00
Name: congestion_surcharge, dtype: float64

In [60]:
df_imputed.congestion_surcharge.fillna(0, inplace=True)

In [61]:
df_imputed.congestion_surcharge.describe()

count    3.066718e+06
mean     2.221034e+00
std      8.366386e-01
min     -2.500000e+00
25%      2.500000e+00
50%      2.500000e+00
75%      2.500000e+00
max      2.500000e+00
Name: congestion_surcharge, dtype: float64

In [63]:
df_imputed.Airport_fee.describe()

count    2.994975e+06
mean     1.074082e-01
std      3.556507e-01
min     -1.250000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.250000e+00
Name: Airport_fee, dtype: float64

In [64]:
df_imputed.Airport_fee.fillna(0, inplace=True)

In [65]:
df_imputed.Airport_fee.describe()

count    3.066718e+06
mean     1.048955e-01
std      3.518408e-01
min     -1.250000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.250000e+00
Name: Airport_fee, dtype: float64

### Save dataset

In [ ]:
df_imputed.to_parquet("imputed_dataset.parquet")